In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
#print(train_data)
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
#print(test_data)

train_data_copy = train_data.copy()

train_data_copy.describe()
#print(train_data_copy)

In [4]:
train_data_copy = train_data_copy.astype('int64') 
train_data_copy = train_data_copy.to_numpy()  #converts to numpy
print(train_data_copy)
train_data_copy.shape

#can only run once to conver the dataframe to a numpy array. 
#Else, if you run more than once, the dataframe already became a numpy array on the first run, so can't convert to numpy array again.

I am going to have the neural network train 40000 rows of data, and then test the remaining 2000 and compare predicted output with actual output to get accuracy of model. May change later.

In [57]:
train_features = train_data_copy[0:41800,1:]
#print(train_features)
train_labels = train_data_copy[0:41800,0]
#print(train_labels)

cv_features = train_data_copy[41800:42000,1:]
#print(cv_features)
cv_labels = train_data_copy[41800:42000,0]
#print(cv_labels)

In [58]:
#This will be shape input to NN
print(train_features[0].shape)

In [59]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        ### START CODE HERE ### 
        tf.keras.Input(shape=(784,)),
        Dense(40, activation="relu"),
        Dense(35, activation="relu"),
        Dense(30, activation="relu"),
        Dense(25, activation="relu"),
        Dense(20, activation="relu"),
        Dense(15, activation="relu"),
        Dense(10, activation="linear")
        ### END CODE HERE ### 
    ], name = "my_model" 
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']  #adds accuracy to model
)

history = model.fit(
    train_features,train_labels,
    epochs=100,
    validation_data = (cv_features, cv_labels)
)

In [60]:
test_data_copy = test_data.copy()
test_data_copy = test_data_copy.astype('int64') 
test_data_copy = test_data_copy.to_numpy()  #converts to numpy
#print(test_data_copy)
test_data_copy.shape

test_features = test_data_copy[0:28000,0:]
print(test_features)

In [61]:
predictions = model.predict(test_features)


In [62]:
#image_of_two = cv_features[999]
prediction = model.predict(test_features[27999].reshape(1,784)) #prediction for a row
print(prediction) #get 10 output probabilities

In [63]:
prediction_p = tf.nn.softmax(prediction) #needed if a problem requires a probability
print(prediction_p)

In [64]:
yhat = np.argmax(prediction_p) #return index of largest probability to find which digit is correct
print(f"np.argmax(prediction_p): {yhat}")

In [66]:
#test_features = pd.DataFrame(columnTransformerForTest.fit_transform(test_features))
labels_pred = np.argmax(model.predict(test_features), axis=1)
#print(labels_pred)
#print(len(labels_pred))

In [67]:
indexes = []
for i in range(1, 28001):
    indexes.append(i)
#print(len(indexes))

In [70]:
output = pd.DataFrame({'ImageId': indexes,'Label': labels_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")